<a href="https://colab.research.google.com/github/stephenbeckr/numerical-analysis-class/blob/student/Demos/Ch2_Intersection_GraphingCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding the intersection of two functions

This is the **student branch** version that is more open-ended. See the **master branch** version for answers.

This is something you might have done in high school on a graphing calculator (the particular functions we're using are inspired by this [post about using the TI-84+](https://www.dummies.com/education/graphing-calculators/how-to-find-points-of-intersection-on-the-ti-84-plus/)).

![TI-84 image](https://github.com/stephenbeckr/numerical-analysis-class/raw/master/Demos/img/398635.image0.jpg)

Let's find the intersection of
$$ f(x) = .2(x-2)x(x+4)  \quad\text{and}\quad g(x) = .5 x $$

First, let's plot the functions:
(for plotting, here's a list of [style sheets](https://matplotlib.org/3.3.1/gallery/style_sheets/style_sheets_reference.html) to use with `plt.style.use`, and [general matplotlib cheatsheets](https://github.com/matplotlib/cheatsheets); if you want to make the axes through the origin, like in the TI-84 picture, this [stackoverflow post](https://stackoverflow.com/a/25689340) gives several methods)

In [ ]:
import numpy as np

# == First, define the function f
f = lambda x : ... # TODO

# or, do it this way:
def f(x) :
  ... # TODO

# == Now, define g
g = lambda x :  ... # TODO


Now, make a plot (useful as a sanity check):

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-notebook')

fig, ax = plt.subplots()

ax.plot( ... TODO ... )

# Useful commands:
# np.linspace(...) or np.arange(...)
#ax.set_ylim(-10,10)
#plt.xticks(range(-10,11,5))
#plt.yticks(range(-10,11,5))
#ax.minorticks_on()
#fig.show() # Optional in jupyter

## Now, let's find all the intersections (it seems we have three of them, looking at the plots).

Use the builtin root finder from `scipy` (see [`scipy.optimize` documentation](https://docs.scipy.org/doc/scipy/reference/optimize.html#root-finding)) called `root_scalar`

You can either specify the method you want, or it will pick a default method for you, based on what you supply, e.g.,
- if you supply an **interval** containing the root, using `bracket = [a,b]`, then it will use one of Brent's methods; you can also request using the bisection method
- if you supply an **initial guess** using `x0 = 1.2` (for example), then it will try something like Newton's method. However, you need to give it a bit more information:
  - to actually use **Newton's method** you have to specify a function that computes the derivative, using `fprime = [insert name-of-derivative-function]`
  - you can use the **secant method** which doesn't need the derivative, but then you have to specify a second initial point via `x1 = 1.3` (for example)

Find all 3 roots
- can you print out the values of the intersections $r$ and $f(r)$ ([pyformat.info]( https://pyformat.info/) has nice information on making nicely formatted python print statements)
- can you add these to the plots above?
- does everything look correct?
- were you able to find all 3 roots?

In [ ]:
from scipy.optimize import root_scalar

# ... TODO ...

### Compare accuracy with `roots`

Be careful: despite a similar sounding name, [`numpy.roots`](https://numpy.org/doc/stable/reference/generated/numpy.roots.html) is **only for polynomials**.

With a bit of pen-and-paper manipulation, you can cast the above intersection problem as one of finding the roots of a polynomial.  Do this, and then find the roots via `numpy.roots`.

- How can you evaluate the accuracy of your answer?
- Which method (`scipy.optimize.root_scalar` vs `numpy.roots`) is more accurate?

Things like `np.poly` are useful; see [Python polynomial information](https://numpy.org/doc/stable/reference/routines.polynomials.html)


In [ ]:
# ... TODO ...

## Let's repeat, for a different function

Again, let's pick a polynomial since we can find the true roots by hand

Let's do $$ f(x) = (x-1) \cdot (x-(1+10^{-8}))^2. $$

- Do we expect any conditioning issues?

### Try to find the roots using the polynomial root-finding routines (`np.roots`)

In [ ]:

trueRoots = (1,1+1e-8)
coeff = np.poly( trueRoots  )
f     = lambda x : np.polyval( coeff, x )

r_numpy = np.roots( coeff )
r_numpy

How did it go?

- Try plotting the function (zoom in to near where the roots are)
- Make a table of values of $x$ and $f(x)$. What do you notice?

In [ ]:
# ... TODO ...

## Let's try finding the roots via a generic rootfinding method (i.e., ignoring the fact that we have a polynomial)

Call [`root_scalar`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html#scipy.optimize.root_scalar). 

In [ ]:
# ... TODO ...

- Did that work any better?
